# モニタリング（再現性）

研究の再現性についてモニタリングします。モニタリングでは、[pipeline.json](../../../experiments/pipeline.json)に設定された順番に実験が再現されます。  
以下のセルを上から実行してください。2回目以降の実行では、画面上部に表示される以下のボタンをクリックしてから実行して下さい。  
![UnfreezeBotton](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/sections/images/unfreeze_button.png)

## 1. パイプライン定義ファイルをチェックする

パイプライン定義ファイルとパッケージ群の情報に相違がないか確認します。  
以下のセルを実行し、エラーが起きないことを確認してください。  
エラーが起きた場合は、表示されたリンクに遷移してパイプライン定義ファイルを修正＆エラーが起きなくなるまで再実行を繰り返してください。

In [ ]:
import os
import json
import glob

# 作業ディレクトリに移動
%cd ~/experiments/

# 実験パイプライン定義ファイルの読み込み
pipeline = json.load(open("pipeline.json"))
print("実験パイプライン：" + str(pipeline))

# 実験パッケージ群の読み込み
files = glob.glob("./*/")

experiments = []
for file in files:
    experiments += [os.path.basename(file[:-1])]
print("実験パッケージ　：" + str(experiments))

# 元のディレクトリに移動
%cd -

# 過不足の確認
if set(pipeline) != set(experiments):
    link = "https://jupyter.cs.rcos.nii.ac.jp" + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + "edit/experiments/pipeline.json"
    raise Exception("実験パイプラインと実験パッケージ間で相違があります\n\n以下のURLに遷移して修正してください\n" + link)

## 2. 実験パイプラインを実行する

### - 2.1 実験パイプラインの定義情報を取得する

In [ ]:
import json

# 作業ディレクトリに移動
%cd ~/experiments/

# 実験パイプライン定義ファイルの読み込み
pipeline = json.load(open("pipeline.json"))
print("実験パイプライン：" + str(pipeline))

### - 2.2 実験パッケージを最新化する

In [ ]:
for experiment in pipeline:
    %cd $experiment
    !git pull
    !datalad get -r .
    %cd ../

### - 2.3 実験パイプラインを実行する

In [ ]:
%cd ~/experiments/

import papermill

isSucceeded = True
for experiment in pipeline:
    print("\n============================================================ " + experiment)

    print("\n# 複数回実行に備えて正解データ保管用ディレクトリを削除")
    cmd = "rm -rf " + experiment + "/output_data_correct/" 
    print("$ " + cmd)
    !$cmd

    print("\n# 既存の出力ディレクトリのデータを正解データとして移動")
    cmd = "mv " + experiment + "/output_data/ " + experiment + "/output_data_correct/" 
    print("$ " + cmd)
    !$cmd

    print("\n# 出力ディレクトリの初期化")
    cmd = "mkdir " + experiment + "/output_data/"
    print("$ " + cmd)
    !$cmd
    cmd = "cp " + experiment + "/output_data_correct/.gitkeep " + experiment + "/output_data/"
    print("$ " + cmd)
    !$cmd
    cmd = "cp " + experiment + "/output_data_correct/.ipynb_checkpoints " + experiment + "/output_data/"
    print("$ " + cmd)
    !$cmd

    print("\n実験再現のための環境構築")
    cmd = "pip install -r " + experiment + "/requirements.txt"
    print("$ " + cmd)
    !$cmd

    print("\n実験ソースコードの実行")
    cmd = experiment + "/source/"
    print("$ cd " + cmd)
    %cd $cmd

    papermill.execute_notebook("main.ipynb", "log.ipynb")
    
    cmd = "-"
    print("$ cd " + cmd)
    %cd $cmd

    print("\n差分の摘出")
    cmd = "diff -r " + experiment + "/output_data_correct/ " + experiment + "/output_data/"
    print("$ " + cmd)
    result = !$cmd
    print(result)
    
    if len(result) > 0:
        isSucceeded = False
        raise Exception("出力データが一致しません")

## 3. モニタリング結果をREADMEに反映する

In [ ]:
%cd ~/WORKFLOW/FLOW/
from util.scripts import utils 

utils.reflect_monitoring_results('reproducibility', isSucceeded)

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import papermill as pm

%cd ~/

pm.execute_notebook(
    'WORKFLOW/FLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = 'モニタリング（再現性）', IS_RECURSIVE = False, TO_GIT = True)
)

## 5. ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../base_FLOW.ipynb)からトップページに遷移できます。  